In [1]:
import pandas as pd
from datetime import datetime
import math

In [19]:
data = pd.read_csv("../00_data/merged_all_by_algo.csv", sep=";", decimal=",")
data['datetime'] = pd.to_datetime(data['datetime'])
data = data.set_index('datetime')
data.head(3)

,date,hour,day_ahead_price,intraday_price,price_diff,solar_capacity,solar_generation_actual,wind_capacity,wind_generation_actual,wind_generation_forecast,solar_generation_forecast
datetime,,,,,,,,,,,
2016-01-01 00:00:00,2016-01-01,0,23.86,29.34,5.48,39021,0,44325,8648,6033,0
2016-01-01 01:00:00,2016-01-01,1,22.39,24.28,1.89,39021,0,44325,8579,7421,0
2016-01-01 02:00:00,2016-01-01,2,20.59,25.26,4.67,39021,0,44325,8542,7266,0


In [26]:
data["total_generation_forecast"] = data["wind_generation_forecast"] + data["solar_generation_forecast"]
data["total_generation_actual"] = data["solar_generation_actual"]+data["wind_generation_actual"]
data["wind_forecast_error"] = data["wind_generation_forecast"] - data["wind_generation_actual"]
data["solar_forecast_error"] = data["solar_generation_forecast"] - data["solar_generation_actual"]
data["forecast_error"] = data["total_generation_forecast"] - data["total_generation_actual"]
data["forecast_error_percent"] = data["forecast_error"]/data["total_generation_forecast"]
data["wind_forecast_error_percent"] = data["wind_forecast_error"]/data["wind_generation_forecast"]
data[(data["wind_forecast_error"]==0) & (data["wind_generation_forecast"]== 0)]["wind_forecast_error_percent"] = 0
data["solar_forecast_error_percent"] = data["solar_forecast_error"]/data["solar_generation_forecast"]
data[(data["solar_forecast_error"]==0) & (data["solar_generation_forecast"]== 0)]["solar_forecast_error_percent"] = 0
data["pricediff_percent"] = data["price_diff"]/data["day_ahead_price"]
# utilization
data["total_capacity"] = data["wind_capacity"] + data["solar_capacity"]
data["solar_utilization"] = (data["solar_generation_actual"]/data["solar_capacity"]).astype(float)
data["wind_utilization"] = (data["wind_generation_actual"]/data["wind_capacity"]).astype(float)
data["total_utilization"] = (data["total_generation_actual"]/data["total_capacity"]).astype(float)
print(data.head(5))
# TODO: moving average calculation until time of decision (information must be known at 12 am on day before)
# moving average 30 days of utilization
data["mov_avg_utilization"] = data.total_utilization.rolling(30*24,  min_periods=1).mean()
data["utilization_frac_t-1/mvg_avg"] = data["total_utilization"].shift(24)/data["mov_avg_utilization"]
# moving average 30 days of forecast error
data["mov_avg_30_forecast_error"] = data.forecast_error.rolling(30*24,  min_periods=1).mean()
data["mov_avg_30_forecast_error_percent"] = data.forecast_error_percent.rolling(30*24,  min_periods=1, win_type="triang").mean()
data["mov_avg_30_wind_forecast_error_percent"] = data.wind_forecast_error_percent.rolling(30*24,  min_periods=1, win_type="triang").mean()
data["mov_avg_30_solar_forecast_error_percent"] = data.solar_forecast_error_percent.rolling(30*24,  min_periods=1, win_type="triang").mean()
# mov avg 10 days
data["mov_avg_10_forecast_error_percent"] = data.forecast_error_percent.rolling(10*24,  min_periods=1, win_type="triang").mean()
data["mov_avg_10_wind_forecast_error_percent"] = data.wind_forecast_error_percent.rolling(10*24,  min_periods=1, win_type="triang").mean()
data["mov_avg_10_solar_forecast_error_percent"] = data.solar_forecast_error_percent.rolling(10*24,  min_periods=1, win_type="triang").mean()
# forecast error avg 24 hours
data["mov_avg_1_forecast_error_percent"] = data.forecast_error_percent.rolling(24,  min_periods=1, win_type="triang").mean()
data["mov_avg_1_wind_forecast_error_percent"] = data.wind_forecast_error_percent.rolling(24,  min_periods=1, win_type="triang").mean()
data["mov_avg_1_solar_forecast_error_percent"] = data.solar_forecast_error_percent.rolling(24,  min_periods=1, win_type="triang").mean()

# season and weekday calculation
seasons = range(4)
data["season"] = data.index.map(lambda x: seasons[math.ceil(x.month/3)-1])
data["month"] = data.index.month
data["weekday"] = data.index.weekday
print(data.head(35))

data.to_csv("../00_data/data_with_calc_fields.csv", sep=";", decimal=",")
print("done")

C:\Users\Tabea\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


                           date  hour  day_ahead_price  intraday_price  \
datetime                                                                 
2016-01-01 00:00:00  2016-01-01     0            23.86           29.34   
2016-01-01 01:00:00  2016-01-01     1            22.39           24.28   
2016-01-01 02:00:00  2016-01-01     2            20.59           25.26   
2016-01-01 03:00:00  2016-01-01     3            16.81           20.24   
2016-01-01 04:00:00  2016-01-01     4            17.41           21.94   

                     price_diff  solar_capacity  solar_generation_actual  \
datetime                                                                   
2016-01-01 00:00:00        5.48           39021                        0   
2016-01-01 01:00:00        1.89           39021                        0   
2016-01-01 02:00:00        4.67           39021                        0   
2016-01-01 03:00:00        3.43           39021                        0   
2016-01-01 04:00:00      

done
